In [1]:
import ee
import geopandas as gpd
import pandas as pd

from pph import fpcde

In [2]:
ee.Initialize()

In [3]:
TARGET_ORBIT = 34

In [4]:
aoi_filename = "../data/pph_region.shp"
gdf_aoi = gpd.read_file(aoi_filename)
gdf_aoi.to_crs(4326, inplace=True)

ee_aoi = ee.FeatureCollection(gdf_aoi.__geo_interface__).geometry()

In [5]:
q_date = [(f"{date}-04-01", f"{date}-10-31" ) for date in range(2017, 2022)]

In [6]:
gdfs = [
    fpcde.Sentinel1Extractor().extract_and_process(ee_aoi, date)
    for date in q_date
]


In [18]:
gdf = pd.concat(gdfs)

In [19]:
gdf = gdf[gdf['relativeOrbitNumber_start'] == TARGET_ORBIT]

In [20]:
# unique y:
gdf.y.unique()

array([48.14, 49.63, 49.64, 51.13, 52.63, 52.6 ])

In [21]:
# 52.2 not in the target
# want 52.17
target_y = [49.63, 49.64]

gdf = gdf[gdf.y.isin(target_y)]

cols = [_ for _ in list(gdf.columns) if _ != 'geometry']
gdf = gdf[cols]

file_name = f"../reports/orbit_{TARGET_ORBIT}_{'_'.join(list(map(str, target_y)))}.csv"


In [22]:
pd.DataFrame(gdf).to_csv(file_name)